In [1]:
import pandas as pd
import sqlite3
from datetime import datetime

In [2]:
# Step 1: Define file paths directly
csv_file = "./29092025_Latest_DB/csv/mst_car.csv"   # adjust path to your CSV
sqlite_file = "./29092025_Latest_DB/guide-mts-database.sqlite3"   # adjust path to your SQLite DB

print(f"CSV selected: {csv_file}")
print(f"SQLite selected: {sqlite_file}")

CSV selected: ./29092025_Latest_DB/csv/mst_car.csv
SQLite selected: ./29092025_Latest_DB/guide-mts-database.sqlite3


In [3]:
# Step 2: Load CSV into pandas
df = pd.read_csv(csv_file)
print("\nOriginal CSV Data:")
print(df.head(20))


Original CSV Data:
   CarID         CarName      CarModel   CarNumber    CarSts  \
0      1           Dummy         Dummy       Dummy     Dummy   
1      2     SWIFT DZIRE    2010 L.M.V  MHO1AX6136  Inactive   
2      3         WAGON-R    2011 L.M.V  MHO1BB2662  Inactive   
3      4   HYUNDAI- I 10    2008 L.M.V  MHO1AH0711  Inactive   
4      5  HYUNDAI ACCENT    2005 L.M.V  MHO1PA6793  Inactive   
5      6       HONDA DIO  2013 M/CYCLE  MHO1BJ9200  Inactive   
6      7            AURA    2021 L.M.V  MH01DT5283    Active   
7      8       NEW DZIRE          2022  MH01DX4274    Active   
8      9     NEW WAGON-R          2022  MH01DX7424    Active   

       LastUpdatedDate  
0  2013-08-07 00:00:00  
1  2013-04-11 00:00:00  
2  2013-04-11 00:00:00  
3  2013-04-11 00:00:00  
4  2013-04-11 00:00:00  
5  2013-04-11 00:00:00  
6  2021-08-28 00:00:00  
7  2022-03-05 00:00:00  
8  2022-06-01 00:00:00  


In [4]:
# Step 2.1: Clean headers (strip spaces/quotes) and lowercase
df.columns = df.columns.str.strip().str.replace('"', '').str.lower()

# Step 2.2: Convert all string values in the DataFrame to lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

print(df.head())

   carid         carname    carmodel   carnumber    carsts  \
0      1           dummy       dummy       dummy     dummy   
1      2     swift dzire  2010 l.m.v  mho1ax6136  inactive   
2      3         wagon-r  2011 l.m.v  mho1bb2662  inactive   
3      4   hyundai- i 10  2008 l.m.v  mho1ah0711  inactive   
4      5  hyundai accent  2005 l.m.v  mho1pa6793  inactive   

       lastupdateddate  
0  2013-08-07 00:00:00  
1  2013-04-11 00:00:00  
2  2013-04-11 00:00:00  
3  2013-04-11 00:00:00  
4  2013-04-11 00:00:00  


C:\Users\balot\AppData\Local\Temp\ipykernel_3664\2227521992.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [5]:
# ---- Step 3: Map active/inactive ----
# Map Active/Inactive/Dummy → boolean-like text
# df["is_active"] = df["is_active"].map({
df["carsts"] = df["carsts"].map({
    "active": "true",
    "inactive": "false",
    "dummy": "false"   # treat dummy as inactive
}).fillna("false")  # default if unknown

In [6]:
# Rename columns to match vehicle table schema
df = df.rename(columns={
    "carid": "id",
    "carname": "vehicle_name",
    "carmodel": "vehicle_model",
    "carnumber": "vehicle_number",
    "carsts": "is_active",
    "lastupdateddate": "created_on"
})

In [7]:
# Add columns required by schema
df["vehicle_fuel_type"] = "cng"
df["updated_on"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [8]:
# Reorder columns to match table
df = df[[
    "id", "vehicle_name", "vehicle_model", "vehicle_number",
    "vehicle_fuel_type", "is_active", "created_on", "updated_on"
]]

print("\nTransformed Data:")
print(df.head(10))


Transformed Data:
   id    vehicle_name vehicle_model vehicle_number vehicle_fuel_type  \
0   1           dummy         dummy          dummy               cng   
1   2     swift dzire    2010 l.m.v     mho1ax6136               cng   
2   3         wagon-r    2011 l.m.v     mho1bb2662               cng   
3   4   hyundai- i 10    2008 l.m.v     mho1ah0711               cng   
4   5  hyundai accent    2005 l.m.v     mho1pa6793               cng   
5   6       honda dio  2013 m/cycle     mho1bj9200               cng   
6   7            aura    2021 l.m.v     mh01dt5283               cng   
7   8       new dzire          2022     mh01dx4274               cng   
8   9     new wagon-r          2022     mh01dx7424               cng   

  is_active           created_on           updated_on  
0     false  2013-08-07 00:00:00  2025-10-01 01:34:11  
1     false  2013-04-11 00:00:00  2025-10-01 01:34:11  
2     false  2013-04-11 00:00:00  2025-10-01 01:34:11  
3     false  2013-04-11 00:00:00  20

In [9]:
# Step 4: Insert into SQLite
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()

In [10]:
# Insert data into table
df.to_sql("vehicles", conn, if_exists="append", index=False)


conn.commit()
conn.close()


print("\nData inserted into vehicles table successfully.")


Data inserted into vehicles table successfully.
